In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout,Flatten, Dense
from keras import applications

In [3]:
img_width, img_height = 150,150

train_data_dir = '../dataset/split_data/train/'
validation_data_dir = '../dataset/split_data/validation/'
nb_train_samples = 13447
nb_validation_samples = 4487
epochs = 3
batch_size = 16

In [10]:
def save_bottleneck_features():
    dataget = ImageDataGenerator(rescale=1./255)
    
    #build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    generator = dataget.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, #Only data, no labels
        shuffle=False
    )
    
    #Returns the output of a model
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    
    # save the output as a Numpy array
    np.save(open('bottleneck_features_train.npy','w'), bottleneck_features_train)
    
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    